In [1]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import gc
import datetime
import sys
import matplotlib.pyplot as plt

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [57]:
#特征读取
train_feature = pd.read_csv("train_feature.csv")
test_feature = pd.read_csv("test_feature.csv")
test_a_feature = pd.read_csv("test_a_feature.csv")

train_feature = pd.read_csv("train_feature_v2.csv")
test_feature = pd.read_csv("test_feature_v2.csv")

In [10]:
#不需要进入模型的数据特征名和label名称
id_columns = ["id"]
custom_columns = ["org_label"]
label_columns = ["label"]
timestamp_columns_name = "date"
timestamp_columns = [timestamp_columns_name]
train_columns_drop = id_columns + custom_columns + timestamp_columns + label_columns
test_columns_drop = id_columns + timestamp_columns
train_data_timestamp = 20171020
display(train_columns_drop)
display(test_columns_drop)

['id', 'org_label', 'date', 'label']

['id', 'date']

In [365]:
train_data_timestamp = 20171020
params={
    'booster':'gbtree',           #gbtree gblinear dart
    'num_boost_round':400,
    'eta': 0.05,                   #0.3 [0,1] 步长
    'gamma':0.1,                  #0 [0,:] 树节点分裂的损失减小最小阈值
    'max_depth':7, # 3 10         #6 [0,:] 树的最大深度阈值
    'min_child_weight':1,#1 6     #1 [0,:] 子节点中最小的样本权重和
    'subsample':0.8,              #1 (0,1] 样本抽样比例
    'colsample_bytree':0.8,       #1 (0,1] 特征抽样比例
    'lambda':50,                   #1 l2正则化权重
    'alpha':50,                    #0 l1正则化权重
    'scale_pos_weight':1,         #1 样本平衡调节 sum(negative cases) / sum(positive cases)
    'objective': 'binary:logistic',#分类，结果为逻辑蒂斯
    'eval_metric': 'auc',
    'nthread':12,
    'seed':0,
    'silent':0
    }

################step1 线下训练##################

In [ ]:
filt_feature_data = train_feature.copy()

In [361]:
#####################################################step 3
#5折交叉验证 获得整体样本训练集和测试集的xgb_pre
#将训练集划分成5折，每一折训练集由其余训练集训练一个xgb模型并进行预测
#测试集则由5个xgb模型分别预测并进行平均得到

#model_xgb = sample_data[label_column]
#model_xgb["xgb_pre"] = 0

model_bst = list()
model_result = pd.DataFrame()
train_dataset = filt_feature_data.ix[filt_feature_data[timestamp_columns_name] < train_data_timestamp, :]
verify_dataset = filt_feature_data.ix[filt_feature_data[timestamp_columns_name] >= train_data_timestamp, :]
train_dataset_y0 = train_dataset.loc[train_dataset.label == 0,:]
train_dataset_y1 = train_dataset.loc[train_dataset.label == 1,:]
undersample_prop = (train_dataset_y1.shape[0] * 20 ) / train_dataset_y0.shape[0]

test_y_data = verify_dataset[["label"]]
test_y_data["xgb_pre"] = 0
for i in range(1):
    undersample_y0 = train_dataset_y0.sample(frac=undersample_prop, random_state=i)
    k_fold_train_data = pd.concat([undersample_y0, train_dataset_y1], axis=0)
    
    model_bst.append(do_model_train_and_pre(k_fold_train_data, verify_dataset, params))
    test_pred = verify_dataset[["xgb_pre"]]
    test_y = verify_dataset[["label"]]
    auc = metrics.roc_auc_score(y_true=test_y, y_score=test_pred)
    fpr, tpr, thresholds = metrics.roc_curve(y_true=test_y, y_score=test_pred)
    atec_score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
    model_result.loc[i, "best_score"] = float(model_bst[i].attributes()["best_score"])
    model_result.loc[i, "test_auc"] = auc
    model_result.loc[i, "atec_score"] = atec_score
    
    test_y_data["xgb_pre"] = test_y_data["xgb_pre"] + verify_dataset["xgb_pre"]
    verify_dataset.drop("xgb_pre", inplace=True, axis=1)

test_y_data["xgb_pre"] = test_y_data["xgb_pre"] / 5

test_pred = test_y_data[["xgb_pre"]]
test_y = test_y_data[["label"]]
auc = metrics.roc_auc_score(y_true=test_y, y_score=test_pred)
fpr, tpr, thresholds = metrics.roc_curve(y_true=test_y, y_score=test_pred)
atec_score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
model_result.loc[i + 1, "best_score"] = float(model_bst[i].attributes()["best_score"])
model_result.loc[i + 1, "test_auc"] = auc
model_result.loc[i + 1, "atec_score"] = atec_score
model_result["auc_diff"] = model_result["best_score"] - model_result["test_auc"]
#model_xgb.loc[test_y_data.index,"xgb_pre"] = test_y_data["xgb_pre"]

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


xgb start
[0]	train-auc:0.853411
Will train until train-auc hasn't improved in 50 rounds.
[5]	train-auc:0.907324
[10]	train-auc:0.912766
[15]	train-auc:0.916022
[20]	train-auc:0.926269
[25]	train-auc:0.945627
[30]	train-auc:0.95679
[35]	train-auc:0.959064
[40]	train-auc:0.960105
[45]	train-auc:0.962869
[50]	train-auc:0.963659
[55]	train-auc:0.96527
[60]	train-auc:0.966677
[65]	train-auc:0.968582
[70]	train-auc:0.969529
[75]	train-auc:0.970647
[80]	train-auc:0.971445
[85]	train-auc:0.972136
[90]	train-auc:0.972666
[95]	train-auc:0.973323
[100]	train-auc:0.973842
[105]	train-auc:0.974282
[110]	train-auc:0.974626
[115]	train-auc:0.974888
[120]	train-auc:0.975131
[125]	train-auc:0.975412
[130]	train-auc:0.975672
[135]	train-auc:0.975964
[140]	train-auc:0.976243
[145]	train-auc:0.976481
[150]	train-auc:0.976677
[155]	train-auc:0.97679
[160]	train-auc:0.976951
[165]	train-auc:0.977096
[170]	train-auc:0.977317
[175]	train-auc:0.97744
[180]	train-auc:0.977611
[185]	train-auc:0.977748
[190]	tra

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


xgb build success


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [35]:
model_result

,best_score,test_auc,atec_score,auc_diff
0,0.983805,0.972695,0.413328,0.01111
1,0.983805,0.972695,0.413328,0.01111


In [300]:
#特征评分
xgb.plot_importance(model_bst[0])

feature_score = model_bst[0].get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
feature_score = pd.DataFrame(feature_score)
display(feature_score)

,0,1
0,dismiss_count,626
1,f7,496
2,f82,358
3,f210,330
4,f5,310
5,f6,301
6,f234,285
7,f238,249
8,f85,229
9,f52,226


################step1 线上训练##################

In [203]:
params={
    'booster':'gbtree',           #gbtree gblinear dart
    'num_boost_round':500,
    'eta': 0.05,                   #0.3 [0,1] 步长
    'gamma':0.1,                  #0 [0,:] 树节点分裂的损失减小最小阈值
    'max_depth':7, # 3 10         #6 [0,:] 树的最大深度阈值
    'min_child_weight':1,#1 6     #1 [0,:] 子节点中最小的样本权重和
    'subsample':0.8,              #1 (0,1] 样本抽样比例
    'colsample_bytree':0.8,       #1 (0,1] 特征抽样比例
    'lambda':50,                   #1 l2正则化权重
    'alpha':50,                    #0 l1正则化权重
    'scale_pos_weight':1,         #1 样本平衡调节 sum(negative cases) / sum(positive cases)
    'objective': 'binary:logistic',#分类，结果为逻辑蒂斯
    'eval_metric': 'auc',
    'nthread':12,
    'seed':0,
    'silent':0
    }

In [333]:

#####################################################step 3
#5折交叉验证 获得整体样本训练集和测试集的xgb_pre
#将训练集划分成5折，每一折训练集由其余训练集训练一个xgb模型并进行预测
#测试集则由5个xgb模型分别预测并进行平均得到

#model_xgb = sample_data[label_column]
#model_xgb["xgb_pre"] = 0

model_bst = list()
train_dataset = train_feature.loc[train_feature[timestamp_columns_name] > 20171005, :]
#train_dataset = train_feature_with_ay0.copy()
verify_dataset = test_feature.copy()
#verify_dataset = test_a_feature.copy()

train_dataset_y0 = train_dataset.loc[train_dataset.label == 0,:]
train_dataset_y1 = train_dataset.loc[train_dataset.label == 1,:]
undersample_prop = (train_dataset_y1.shape[0] * 20 ) / train_dataset_y0.shape[0]

test_y_data = verify_dataset[["id"]]
test_y_data["xgb_pre"] = 0
for i in range(5):
    undersample_y0 = train_dataset_y0.sample(frac=undersample_prop, random_state=i)
    k_fold_train_data = pd.concat([undersample_y0, train_dataset_y1], axis=0)
    model_bst.append(do_model_train_and_pre_online(k_fold_train_data, verify_dataset, params))

    test_y_data["xgb_pre"] = test_y_data["xgb_pre"] + verify_dataset["xgb_pre"]
    verify_dataset.drop("xgb_pre", inplace=True, axis=1)
test_y_data["xgb_pre"] = test_y_data["xgb_pre"] / 5
#model_xgb.loc[test_y_data.index,"xgb_pre"] = test_y_data["xgb_pre"]

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


xgb start
[0]	train-auc:0.843659
Will train until train-auc hasn't improved in 50 rounds.
[5]	train-auc:0.882249
[10]	train-auc:0.944151
[15]	train-auc:0.959324
[20]	train-auc:0.961214
[25]	train-auc:0.963701
[30]	train-auc:0.964756
[35]	train-auc:0.967975
[40]	train-auc:0.968964
[45]	train-auc:0.969941
[50]	train-auc:0.971065
[55]	train-auc:0.973437
[60]	train-auc:0.974219
[65]	train-auc:0.974766
[70]	train-auc:0.975272
[75]	train-auc:0.976017
[80]	train-auc:0.976822
[85]	train-auc:0.977234
[90]	train-auc:0.977717
[95]	train-auc:0.978042
[100]	train-auc:0.978241
[105]	train-auc:0.978529
[110]	train-auc:0.978736
[115]	train-auc:0.978962
[120]	train-auc:0.979277
[125]	train-auc:0.979424
[130]	train-auc:0.979626
[135]	train-auc:0.979751
[140]	train-auc:0.979893
[145]	train-auc:0.980083
[150]	train-auc:0.980178
[155]	train-auc:0.980316
[160]	train-auc:0.980414
[165]	train-auc:0.980527
[170]	train-auc:0.980642
[175]	train-auc:0.980737
[180]	train-auc:0.980919
[185]	train-auc:0.981036
[190]

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


xgb start
[0]	train-auc:0.851278
Will train until train-auc hasn't improved in 50 rounds.
[5]	train-auc:0.874512
[10]	train-auc:0.945196
[15]	train-auc:0.9607
[20]	train-auc:0.961773
[25]	train-auc:0.964257
[30]	train-auc:0.965654
[35]	train-auc:0.968072
[40]	train-auc:0.96937
[45]	train-auc:0.970461
[50]	train-auc:0.971554
[55]	train-auc:0.97231
[60]	train-auc:0.973958
[65]	train-auc:0.974642
[70]	train-auc:0.97513
[75]	train-auc:0.975862
[80]	train-auc:0.976655
[85]	train-auc:0.977129
[90]	train-auc:0.977641
[95]	train-auc:0.977926
[100]	train-auc:0.978231
[105]	train-auc:0.978551
[110]	train-auc:0.978748
[115]	train-auc:0.978932
[120]	train-auc:0.979101
[125]	train-auc:0.979329
[130]	train-auc:0.979515
[135]	train-auc:0.979614
[140]	train-auc:0.979721
[145]	train-auc:0.979889
[150]	train-auc:0.980055
[155]	train-auc:0.980196
[160]	train-auc:0.980325
[165]	train-auc:0.980474
[170]	train-auc:0.98066
[175]	train-auc:0.980764
[180]	train-auc:0.980903
[185]	train-auc:0.980984
[190]	train

[85]	train-auc:0.977107
[90]	train-auc:0.977596
[95]	train-auc:0.977873
[100]	train-auc:0.978157
[105]	train-auc:0.978479
[110]	train-auc:0.97872
[115]	train-auc:0.978968
[120]	train-auc:0.979163
[125]	train-auc:0.979318
[130]	train-auc:0.979523
[135]	train-auc:0.979633
[140]	train-auc:0.97977
[145]	train-auc:0.97993
[150]	train-auc:0.980106
[155]	train-auc:0.980224
[160]	train-auc:0.980344
[165]	train-auc:0.980491
[170]	train-auc:0.980622
[175]	train-auc:0.980739
[180]	train-auc:0.980874
[185]	train-auc:0.980952
[190]	train-auc:0.981067
[195]	train-auc:0.981205
[200]	train-auc:0.981306
[205]	train-auc:0.981373
[210]	train-auc:0.981461
[215]	train-auc:0.981535
[220]	train-auc:0.981593
[225]	train-auc:0.981649
[230]	train-auc:0.981702
[235]	train-auc:0.981779
[240]	train-auc:0.981839
[245]	train-auc:0.9819
[250]	train-auc:0.981964
[255]	train-auc:0.982022
[260]	train-auc:0.982068
[265]	train-auc:0.982133
[270]	train-auc:0.98217
[275]	train-auc:0.982246
[280]	train-auc:0.982297
[285]	tra

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
#输出测试集评分
test_y_data.rename(columns={"xgb_pre":"score"}, inplace=True)
test_b_result = test_y_data.copy()
test_b_result.to_csv("test_b_result.csv", index=False)

In [250]:
#提取a榜数据中的黑样本加入原始训练集
#插入的变量和训练集的变量位置对应，否则模型预测报错，日期映射到10月份
#控制a榜的黑白样本比例和原始训练集一致（0.012）此时的概率值为0.52
#6025/(491668 - 6025)  12116/(993941 - 12116)
test_a_sample_y0_id = test_a_result.loc[test_a_result.score>0.8,["id"]]
test_a_sample_y0 = pd.merge(test_a_sample_y0_id, test_a_feature, how="left", on="id")
test_a_sample_y0.insert(1, "org_label",2)
test_a_sample_y0.insert(3, "label",1)
test_a_sample_y0.date = test_a_sample_y0.date - (20180105-20171005)
train_feature_with_ay0 = pd.concat([train_feature, test_a_sample_y0], axis=0)

In [ ]:
data_path = "D:/fangshengjie/jupyter_workspace/atec/data/"
test_a_data_path = data_path + "atec_anti_fraud_test_a.csv"
test_a_data = pd.read_csv(test_a_data_path)

In [286]:
test_a_data_y1 = pd.merge(test_a_sample_y0_id, test_a_data, how="left", on="id")
test_a_data_y1.insert(1, "label",1)
test_a_data_y1.insert(300, "org_label",2)
test_a_data_y1.date = test_a_data_y1.date - (20180105-20171005)
test_a_data_y1.to_csv("test_a_data_y1.csv", index=False)

In [64]:
#输出测试集评分
test_y_data.rename(columns={"xgb_pre":"score"}, inplace=True)
test_a_result = test_y_data.copy()
test_a_result.to_csv("test_a_result.csv", index=False)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


################模型训练接口##################

In [11]:
def do_model_train_and_pre(k_fold_train_data, k_fold_test_data, params):
    train_x = k_fold_train_data.drop(labels=train_columns_drop, axis=1)
    train_y = k_fold_train_data["label"]
    dtrain = xgb.DMatrix(data=train_x, label=train_y)  
    print("xgb start")
    # del train_x
    # del train_y
    # del test_x
    # gc.collect()

    watchlist = [(dtrain,'train')]
    bst = xgb.train(params=params, 
                    dtrain=dtrain, 
                    num_boost_round=params["num_boost_round"], 
                    evals=watchlist,
                    verbose_eval=5,
                    early_stopping_rounds=50)
    
    test_x = k_fold_test_data.drop(labels=train_columns_drop, axis=1)
    test_y = k_fold_test_data.loc[:,["label"]]
    dtest = xgb.DMatrix(data=test_x)
    test_pred = bst.predict(data=dtest)
    k_fold_test_data["xgb_pre"] = test_pred
#     auc = metrics.roc_auc_score(y_true=test_y, y_score=test_pred)
#     fpr, tpr, thresholds = metrics.roc_curve(y_true=test_y, y_score=test_pred)
#     score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
#     print("atec_score:",score)
#     print("auc:",auc)
    print("xgb build success")
    gc.collect()
    return bst

In [12]:
def do_model_train_and_pre_online(k_fold_train_data, k_fold_test_data, params):
    train_x = k_fold_train_data.drop(labels=train_columns_drop, axis=1)
    train_y = k_fold_train_data["label"]
    dtrain = xgb.DMatrix(data=train_x, label=train_y)
    print("xgb start")
    # del train_x
    # del train_y
    # del test_x
    # gc.collect()

    watchlist = [(dtrain,'train')]
    bst = xgb.train(params=params, 
                    dtrain=dtrain, 
                    num_boost_round=params["num_boost_round"], 
                    evals=watchlist,
                    verbose_eval=5,
                    early_stopping_rounds=50)
    
    #dtest = xgb.DMatrix(data=test_x)
    #test_x = k_fold_test_data.drop(labels=train_columns_drop, axis=1)
    test_x = k_fold_test_data.drop(labels=test_columns_drop, axis=1)
    #test_y = k_fold_test_data.loc[:,["label"]]
    dtest = xgb.DMatrix(data=test_x)
    test_pred = bst.predict(data=dtest)
    k_fold_test_data["xgb_pre"] = test_pred
#     print(test_y.shape)
#     print(test_pred.shape)
#     auc = metrics.roc_auc_score(y_true=test_y, y_score=test_pred)
#     fpr, tpr, thresholds = metrics.roc_curve(y_true=test_y, y_score=test_pred)
#     score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
#     print("atec_score:",score)
#     print("auc:",auc)
    print("xgb build success")
    gc.collect()
    
    #预测k折交叉验证的测试集（1/k）
#     test_feature_data = k_fold_test_data.drop(labels=test_columns_drop, axis=1)
#     dtest = xgb.DMatrix(data=test_feature_data)
#     k_fold_test_pred = bst.predict(data=dtest)
#     k_fold_test_data["xgb_pre"] = k_fold_test_pred
#     #预测整体样本的测试集
#     dtest = xgb.DMatrix(data=test_data)
#     test_data_pred = bst.predict(data=dtest)
#     test_data["xgb_pre"] = test_data_pred
    return bst